In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
pal = sns.color_palette()
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
train_qs = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist()).astype(str)
test_qs = pd.Series(df_test['question1'].tolist() + df_test['question2'].tolist()).astype(str)
q1 = pd.Series(df_train['question1'][0]) #para sacar la primera pregunta de question1
print(q1)
dist_train = train_qs.apply(len)
dist_test = test_qs.apply(len)
#plt.figure(figsize=(15, 10))
#plt.hist(dist_train, bins=200, range=[0, 200], color=pal[2], normed=True, label='train')
#plt.hist(dist_test, bins=200, range=[0, 200], color=pal[1], normed=True, alpha=0.5, label='test')
#plt.title('Normalised histogram of character count in questions', fontsize=15)
#plt.legend()
#plt.xlabel('Number of characters', fontsize=15)
#plt.ylabel('Probability', fontsize=15)

#print('mean-train {:.2f} std-train {:.2f} mean-test {:.2f} std-test {:.2f} max-train {:.2f} max-test {:.2f}'.format(dist_train.mean(), 
#                          dist_train.std(), dist_test.mean(), dist_test.std(), dist_train.max(), dist_test.max()))

In [ ]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R
#print(stops)

In [ ]:
train_word_match = df_train.apply(word_match_share, axis=1, raw=True)
decision = {} #tabla con los datos estimados, duplicados o no.
acierto =  {}#tabla para porcentajes de aciertos en la estimación
duplicados = df_train['is_duplicate'] #tabla con los datos de pares duplicados
len(df_train)
#print(duplicados)

In [ ]:
for index in range(len(train_word_match)):
    if (train_word_match[index] > 0.35):
        decision[index] = 1
    else:
        decision[index] = 0
#print (decision) 
counter = 0
for index in range(len(train_word_match)):
    if (decision[index] == duplicados[index]):
        acierto[index]=1
        counter = counter +1
    else:
        acierto[index]=0
        
porcentaje = (counter*100)/404290
print(porcentaje)
print(len(decision))

In [ ]:
from collections import Counter

# If a word appears only once, we ignore it completely (likely a typo)
# Epsilon defines a smoothing constant, which makes the effect of extremely rare words smaller
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

eps = 5000 
words = (" ".join(train_qs)).lower().split()
words_q1 = (" ".join(q1)).lower().split()
for w in range(len(words_q1)):
    print(words_q1[w])
 

    
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}#estudiar bien que hace

In [ ]:
#df_train['q1_n_words'] = df_train['question1'].apply(lambda row: len(row.split(" ")))
#df_train['q2_n_words'] = df_train['question2'].apply(lambda row: len(row.split(" ")))

stops = set(stopwords.words("english"))
ratiow = []

for i in range(404290):
    n_words = 0
    q1 = pd.Series(df_train['question1'][i]).astype(str) #para sacar la primera pregunta de question1
    q2 = pd.Series(df_train['question2'][i]).astype(str)
    words_q1 = (" ".join(q1)).lower().split()
    words_q2 = (" ".join(q2)).lower().split()
    media = (len(words_q1) + len(words_q2))/2
    minimo = min(len(words_q1), len(words_q2))
    for w1 in words_q1:
        for w2 in words_q2:
            if (w1==w2) and (w1 not in stops):
                n_words = n_words + 1
    #print(n_words)            
    r = n_words/minimo
    ratiow.append(r)

for w in range(20):
    print(ratiow[w])
    
#print(len(ratiow))                
df_train.head()                                   
                

In [ ]:
stops = set(stopwords.words("english"))


def same_word_ratio(row):
    
    ratiow = []
    for i in range(404290):
        n_words = 0
        q1 = pd.Series(row['question1'][i]).astype(str) #para sacar la primera pregunta de question1
        q2 = pd.Series(row['question2'][i]).astype(str)
        words_q1 = (" ".join(q1)).lower().split()
        words_q2 = (" ".join(q2)).lower().split()
        media = (len(words_q1) + len(words_q2))/2
        minimo = min(len(words_q1), len(words_q2))
        for w1 in words_q1:
            for w2 in words_q2:
                if (w1==w2) and (w1 not in stops):
                    n_words = n_words + 1
        #print(n_words)            
        r = n_words/minimo
        ratiow.append(r)
    return ratiow
    
    

In [ ]:
decision2 = {} #tabla con los datos estimados, duplicados o no.

acierto2 =  {} #tabla para porcentajes de aciertos en la estimación
duplicados2 = df_train['is_duplicate'] #tabla con los datos de pares duplicados
#print(duplicados)
ratiow = same_word_ratio(df_train)#llamada a la funcion hecha


for index in range(len(ratiow)):
    if (ratiow[index] > 0.25):
        decision2[index] = 1
    else:
        decision2[index] = 0
#print (decision) 
counter = 0
for index in range(len(ratiow)):
    if (decision2[index] == duplicados2[index]):
        acierto2[index]=1
        counter = counter +1
    else:
        acierto2[index]=0
        
porcentaje = (counter*100)/404290 
print(porcentaje)

In [ ]:
decision2_test = {}
ratiow_test = same_word_ratio(df_test)

for index in range(len(ratiow_test)):
    if (ratiow_test[index] > 0.25):
        decision2_test[index] = 1
    else:
        decision2_test[index] = 0

for w in range(20):
    print(decision2_test[w])

In [ ]:
print('Most common words and weights: \n')
print(sorted(weights.items(), key=lambda x: x[1] if x[1] > 0 else 9999)[:10])
print('\nLeast common words and weights: ')
(sorted(weights.items(), key=lambda x: x[1], reverse=True)[:10])

In [ ]:
def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [ ]:
tfidf_train_word_match = df_train.apply(tfidf_word_match_share, axis=1, raw=True)
decision_tfidf = {} #tabla con los datos estimados, duplicados o no.
acierto_tfidf =  {} #tabla para porcentajes de aciertos en la estimación
duplicados_tfidf = df_train['is_duplicate'] #tabla con los datos de pares duplicados
print(duplicados_tfidf)

In [ ]:
for index in range(len(tfidf_train_word_match)):
    if (tfidf_train_word_match[index] > 0.35):
        decision_tfidf[index] = 1
    else:
        decision_tfidf[index] = 0
#print (decision_tfidf) 

counter = 0
for index in range(len(tfidf_train_word_match)):
    if (decision_tfidf[index] == duplicados_tfidf[index]):
        acierto_tfidf[index]=1
        counter = counter +1
    else:
        acierto_tfidf[index]=0
        
porcentaje_tfidf = (counter*100)/404265 
print(porcentaje_tfidf)

In [ ]:
word_match_test = df_test.apply(word_match_share, axis=1, raw=True)
decision_test = {} #tabla con los datos estimados, duplicados o no.
#duplicados = df_train['is_duplicate'] #tabla con los datos de pares duplicados
len(df_test)
#print(duplicados)

In [ ]:
for index in range(len(word_match_test)):
    if (word_match_test[index] > 0.35):
        decision_test[index] = 1
    else:
        decision_test[index] = 0
#print (decision) 
for w in range(20):
    print(decision_test[w])
len(decision_test)

In [ ]:
submission = pd.DataFrame({'is_duplicate': decision2_test, 'test_id': df_test['test_id']})
submission.to_csv("submission.csv", index=False)
#sub = pd.DataFrame()
#sub['test_id'] = df_test['test_id']
#sub['is_duplicate'] = decision_test
#sub.to_csv('simple_counw.csv', index=False)
submission.head()